##**Installation of TwelveLabs SDKs**

In [3]:
!pip install -U -q twelvelabs
from twelvelabs import TwelveLabs
from twelvelabs.models.task import Task

##**Configure your API key. Add API Key as a env variable.**
Signup for TwelveLabs and Get your API keys [here](https://playground.twelvelabs.xyz/dashboard/api-key). No credit card is required to use the Free plan. The Free plan includes indexing of 600 mins of videos, which is enough for a small project.

*To run the following cell in a*
* *Colab Notebook, your API key must be stored it in a Colab Secret named TL_API_KEY. If you don't already have an API key, or you're not sure how to create a Colab Secret, see [this](https://medium.com/@parthdasawant/how-to-use-secrets-in-google-colab-450c38e3ec75) for an example.*
* *In other Python environments, You can also use Python variables using `os.environ()`*



In [5]:
from google.colab import userdata
TL_API_KEY=userdata.get('TL_API_KEY')

## **Step:1 - Create Index (You can also use an existing Index)**
Two main inputs needed for Index creation is the Engine Type and Engine Options. Learn more [here](https://docs.twelvelabs.io/docs/create-indexes).

**Twelve Labs provides two distinct engine types** - embedding and generative, each serving unique purposes in multimodal video understanding.

* Embedding engines (Marengo) : `marengo2.6` engine is proficient at performing tasks such as search and classification, enabling enhanced video understanding.
* Generative engines (Pegasus): `pegasus1` engine generates text based on your videos.
Learn more [here](https://docs.twelvelabs.io/docs/video-understanding-engines)

> *Video Duration limits: For Marengo, it must be between 4 seconds and 2 hours (7,200s). For Pegasus, it must be between 4 seconds and 20 minutes (1200s). In a future release, the maximum duration for Pegasus will be 2 hours (7,200 seconds).*


You can use both engines on the same index but the limits of Pegasus will apply. E.g. If you create an Index with both `Marengo` and `Pegasus` then you will be limited to videos with maximum duration of 20mins.

**Engine options** indicate the types of information a video understanding engine will process. When creating a new index, you must specify the engines and the associated engines options that you want to enable.

The following engine options are available:

* `visual`: This option allows you to analyze video content as you would see and hear from it, including actions, objects, sounds, and events, and excluding human speech.
* `conversation`: This option allows you to analyze human speech within your videos.
* `text_in_video`: This option allows you to detect and extract text (OCR) shown within your videos. *This option is not available for `Pegasus` Engine type.*
* `logo`: This option allows you to identify the presence of brand logos within your videos. *This option is not available for `Pegasus` Engine type.*

In [6]:
import os
from glob import glob
# The path to the directory containing the videos you wish to upload.
VIDEO_PATH = "men-whoa-whoa-whoa-s97.mp4"  # In a colab Notebook, just upload the file in the root folder and just give the filename.

client = TwelveLabs(api_key=TL_API_KEY)

index = client.index.create(
    name="Temp-1",
    engines=[
        {
            "name": "marengo2.6",
            "options": ["visual", "conversation", "text_in_video"],
        }
    ],
    addons=["thumbnail"], # Optional
)
print(f"Created index: id={index.id} name={index.name} engines={index.engines}")


Created index: id=6660dd8ce6fb7df29de0e589 name=Temp-1 engines=root=[Engine(name='marengo2.6', options=['visual', 'conversation', 'text_in_video'], addons=None)]


## **Step:2 - Upload videos to an Index**

The video you wish to upload must meet the following requirements:
* Video resolution: Must be greater or equal than 360p and less or equal than 4K. For consistent search results, Twelve Labs recommends you upload 360p videos.
* Video and audio formats: The video files you wish to upload must be encoded in the video and audio formats listed on the FFmpeg Formats Documentation page. For videos in other formats, contact us at support@twelvelabs.io.
* Duration: For Marengo, it must be between 4 seconds and 2 hours (7,200s). For Pegasus, it must be between 4 seconds and 20 minutes (1200s). In a future release, the maximum duration for Pegasus will be 2 hours (7,200 seconds).
* File size: Must not exceed 2 GB.
* Audio track: If the conversation engine option is selected, the video you're uploading must contain an audio track.

Learn more in [docs](https://docs.twelvelabs.io/docs/upload-single-videos)

In [7]:
video_files = glob(VIDEO_PATH)
if(len(video_files)==0):
  raise Exception("Video file was not provided")
for video_file in video_files:
  print(f"Uploading {video_file}")
  task = client.task.create(index_id=index.id, file=video_file, language="en")
  print(f"Created task: id={task.id}")

  # (Optional) Monitor the video indexing process
  # Utility function to print the status of a video indexing task
  def on_task_update(task: Task):
          print(f"  Status={task.status}")
  task.wait_for_done(sleep_interval=50, callback=on_task_update)
  if task.status != "ready":
      raise RuntimeError(f"Indexing failed with status {task.status}")
  print(f"Uploaded {video_file}. The unique identifer of your video is {task.video_id}.")


Uploading men-whoa-whoa-whoa-s97.mp4
Created task: id=6660dd9be6fb7df29de0e58a
  Status=pending
  Status=pending
  Status=indexing
  Status=ready
Uploaded men-whoa-whoa-whoa-s97.mp4. The unique identifer of your video is 6660dd9cd22b3a3c97bf134c.


## **Step:3 - Search By Text**
*Soon, you can also perform search by an Image.*

In [11]:
#Search Query
search_results = client.search.query(index_id=index.id, query="A green color person talking", options=["visual", "conversation"])

# Utility function to print a specific page
def print_page(page):
  for clip in page:
    print(
        f" video_id={clip.video_id} score={clip.score} start={clip.start} end={clip.end} confidence={clip.confidence}"
    )
print_page(search_results.data)
while True:
    try:
        print_page(next(search_results))
    except StopIteration:
        break

 video_id=6660dd9cd22b3a3c97bf134c score=69.0 start=0.0 end=5.0 confidence=low
